In [1]:
from Datos import Datos
from Arm import ArmkNN
import numpy as np

from sklearn.model_selection import train_test_split

dataset = Datos("ratings.csv")

kNNcols = ['userId','movieId','rating']
kNNdataset = dataset.extraeCols(kNNcols)

listUsers = np.unique(kNNdataset[:,0])
knn = ArmkNN(20)

In [2]:
knn.rec_item(kNNdataset,1)

2858.0

In [ ]:
# Prueba de kNN
train, test = train_test_split(kNNdataset, train_size=0.1)

num = 0
den = 0

for u in listUsers:
    item = knn.rec_item(train, u)
    # Ejemplo encontrado para la recomendación
    mask1 = test[:,0] == u
    mask2 = test[:,1] == item
    mask = np.logical_and(mask1,mask2)
    test, hit = test[np.logical_not(mask)], test[mask]
    if(len(hit) != 0):
        train = np.vstack((train,hit))
        den += 1
        if(hit[0,2] >= 3):
            num += 1
        print(num/den)
    else:
        train = np.vstack((train,[u,item,1.]))

1.0
1.0
0.6666666666666666
0.75
0.8
0.8333333333333334
0.8571428571428571
0.75
0.7777777777777778
0.8
0.8181818181818182
0.8333333333333334
0.8461538461538461
0.7857142857142857
0.8
0.8125
0.8235294117647058
0.8333333333333334
0.8421052631578947
0.85
0.8571428571428571
0.8636363636363636
0.8695652173913043
0.875
0.88
0.8846153846153846
0.8888888888888888
0.8928571428571429
0.896551724137931
0.9
0.9032258064516129
0.90625
0.9090909090909091
0.9117647058823529
0.9142857142857143
0.9166666666666666
0.918918918918919
0.9210526315789473
0.8974358974358975
0.9
0.9024390243902439
0.9047619047619048
0.9069767441860465
0.9090909090909091
0.8888888888888888
0.8913043478260869
0.8936170212765957
0.8958333333333334
0.8979591836734694
0.9
0.8823529411764706
0.8846153846153846
0.8867924528301887
0.8888888888888888
0.8909090909090909
0.8928571428571429
0.8947368421052632
0.896551724137931
0.8983050847457628
0.9
0.9016393442622951
0.9032258064516129
0.9047619047619048
0.90625
0.9076923076923077
0.9090